## 193 에코 서버 만들기
- 네트워크로 메시지를 수신하여 메시지 송신자에게 수신한 메세지를 그대로 돌려보내는 서버 프로그램을 작성해보면서 소켓프로그래밍에 대해 이해

In [ ]:
import socket

HOST = ''
PORT = 9009

def runserver():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.bind((HOST,PORT))
        sock.listen(1)
        print('클라이언트 연결을 기다리는중')
        conn, addr = sock.accept()
        with conn:
            print(f'{addr[0]}과 연결됨')
            while conn:
                data = conn.recv(1024)
                if not data :
                    break
                print(f'메세지 수신{data.decode()}')
                conn.sendall(data)
runserver()

클라이언트 연결을 기다리는중


## 194 에코 클라이언트 만들기
- 네트워크로 메시지를 서버로 송신하고 서버로부터 메시지를 그대로 수신하는 클라이언트
- 193을 실행 후 194를 명령프롬프트로 실행하여 메세지를 보내면 됨

In [ ]:
import socket

HOST = 'localhost'
PORT = 9009

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock :
    sock.connect((HOST, PORT))
    msg = input('메시지 입력 : ')
    sock.sendall(msg.encode())
    data = sock.recv(1024)

print(f'에코 서버로 받은 데이터 [{data.decode()}]')

## 195 에코서버 만들기
- socketserver를 이용하여 기능이 향상된 에코서버를 만듬

In [12]:
import socketserver

HOST = ''
PORT = 9009

class MyTcpHandler(socketserver.BaseRequestHandler):
    # 이 클래스는 서버 하나당 단 한번 초기화 됩니다.
    # handle 메소드에 클라이언트 연결 처리를 위한 로직을 구현합니다.
    
    def handle(self):
        print(f'[{self.client_address[0]}에 연결됨]')
        
        try :
            while True:
                self.data = self.request.recv(1024)
                if self.data.decode() == '/quit':
                    print(f'[{self.client_address[0]}]사용자에 의해 중단')
                    return
                
                print(f'[{self.data.decode()}]')
                self.request.sendall(self.data)
        except Exception as e:
            print(e)
            
def runserver():
    print('+++에코 서버를 시작합니다.')
    print('+++에코 서버를 끝내려면 Ctrl + C를 누르세요.')

    try:
        server = socketserver.TCPServer((HOST,PORT), MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt :
        print('+++ 에코 서버를 종료 합니다.')
            
runserver()

+++에코 서버를 시작합니다.
+++에코 서버를 끝내려면 Ctrl + C를 누르세요.
[127.0.0.1에 연결됨]
[안녕하세요]
[하이루]
[오 계삭나옴]
[신기방기]
[quit]
[127.0.0.1]사용자에 의해 중단
+++ 에코 서버를 종료 합니다.


## 196 에코 클라이언트 만들기
- 사용자가 '/quit'를 입력할때까지 동작하는 에코 클라이언트를 작성
- 195 에코서버를 실행하고 명령프롬프트에서 196을 실행하면 됨.

In [10]:
import socket

HOST = 'localhost'
PORT = 9009

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
    sock.connect((HOST, PORT))
    
    while True:
        msg = input('메시지 입력하세요 : ')
        if msg == '/quit':
            sock.sendall(msg.encode())
            break

        sock.sendall(msg.encode())
        data = sock.recv(1024)
        print(f'에코 서버로 부터 받은 데이터[{data.decode()}]')
    
print('클라이언트 종료')

## 197 파일 송신 프로그램 만들기
- 클라이언트가 요청하는 파일을 클라이언트로 전송하는 서버 프로그램을 작성

In [18]:
import socketserver
from os.path import exists

HOST =''
PORT = 9009

class MyTcpHandler(socketserver.BaseRequestHandler):
    def handle(self):
        data_trasferred = 0
        print(f'연결 됨 : [{self.client_address[0]}]')
        filename = self.request.recv(1024)
        filename = filename.decode()
        
        if not exists(filename):
            return
        
        print(f'파일 [{filename}] 전송 시작..')
        with open(filename, 'rb') as f:
            try :
                data = f.read(1024)
                while data :
                    data_trasferred +=self.request.send(data)
                    data = f.read(1024)
                    
            except Exception as e:
                print(e)
                
        print(f'전송완료 : [{filename}], 전송량 : [{data_trasferred}]')

def runserver():
    print('+++ 파일 서버를 시작합니다.')
    print('+++ 파일 서버를 끝내려면 Ctrl + C를 누르세요.')
    
    try:
        server = socketserver.TCPServer((HOST,PORT), MyTcpHandler)
        server.serve_forever()
    except KeyboardInterrupt :
        print('+++ 파일 서버를 종료합니다.')
        
runserver()

+++ 파일 서버를 시작합니다.
+++ 파일 서버를 끝내려면 Ctrl + C를 누르세요.
연결 됨 : [127.0.0.1]
연결 됨 : [127.0.0.1]
파일 [access_log.txt] 전송 시작..
전송완료 : [access_log.txt], 전송량 : [50406]
+++ 파일 서버를 종료합니다.


## 198 파일 수신 프로그램 만들기
- 서버로부터 파일을 다운로드 받는 클라이언트 프로그램을 작성

In [19]:
import socket

HOST = 'localhost'
PORT = 9009

def getfilefromserver(filename):
    data_trasferred = 0
    
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        sock.sendall(filename.encode())
        
        data = sock.recv(1024)
        if not data :
            print(f'{filename} : 서버에 존재하지 않거나 전송중 오류 발생')
            return
        
        with open('download/' + filename, 'wb') as f:
            try:
                while data:
                    f.write(data)
                    data_trasferred += len(data)
                    data = sock.recv(1024)
            except Exception as e:
                print(e)
                
    print(f'파일[{filename} 전송종료, 전송량[{data_trasferred}]]')

filename = input('다운로드 받을 파일 이름을 입력하세요 :')
getfilefromserver(filename)

## 199 채팅 서버 만들기
- 동시에 다수의 사용자가 대화 가능한 간단한 채팅 서버를 만들기

In [28]:
import socketserver
import threading

HOST = ''
PORT = 9009
lock = threading.Lock()

class UserManager:
    def __init__(self):
        self.users = {}
    
    def adduser(self, username, conn, addr):
        if username in self.users:
            conn.send('이미 등록된 사용자 입니다.\n'.encode())
            return None
        
        # 새로운 사용자를 등록
        lock.acquire()
        self.users[username] = (conn, addr)
        lock.release()
        
        self.sendmessagetoall(f'[{username}]님이 입장하셨습니다.')
        print(f'+++ 대화 참여자 수 : [{len(self.users)}]')
        
        return username
    
    def removeuser(self, username):
        if username not in self.users:
            return
        lock.acquire()
        del self.users[username]
        lock.release()
        
        self.sendmessagetoall(f'[{username}]님이 퇴장했습니다.')
        print(f'+++ 대화 참여자 수 : [{len(self.users)}]')
        
    def messagehandler(self, username, msg):
        if msg[0] != '/':
            self.sendmessagetoall(f'[{username}] : {msg}')
            return
        
        if msg.strip() == '/quit':
            self.removeuser(username)
            return -1
        
    def sendmessagetoall(self, msg):
        for conn, addr in self.users.values():
            conn.send(msg.encode())
            
class MyTcpHandler(socketserver.BaseRequestHandler):
    userman = UserManager()
    
    def handle(self):
        print(f'[{self.client_address[0]}]에 연결됨')
        
        try:
            username = self.registerusername()
            msg = self.request.recv(1024)
            while msg:
                print(msg.decode())
                if self.userman.messagehandler(username, msg.decode()) == -1:
                    self.request.close()
                    break
                msg = self.request.recv(1024)
        except Exception as e:
            print(e)
            
        print(f'[{self.client_address[0]}] 접속 종료')
        self.userman.removeuser(username)
        
    def registerusername(self):
        while True:
            self.request.send('로그인ID:'.encode())
            username = self.request.recv(1024)
            username = username.decode().strip()
            if self.userman.adduser(username, self.request, self.client_address):
                return username
            
class ChatingServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

def runserver():
    print('+++채팅 서버를 시작합니다.')
    print('+++채팅 서버를 종료하려면 Ctrl + C를 누르세요.')
    
    try :
        server = ChatingServer((HOST, PORT), MyTcpHandler)
        server.serve_forever()
        
    except KeyboardInterrupt:
        print('+++채팅 서버를 종료 합니다.')
        server.shutdown()
        server.server_close()
        
runserver()

+++채팅 서버를 시작합니다.
+++채팅 서버를 종료하려면 Ctrl + C를 누르세요.
[127.0.0.1]에 연결됨
+++ 대화 참여자 수 : [1]
하이요
[127.0.0.1]에 연결됨
+++ 대화 참여자 수 : [2]
하이
[127.0.0.1]에 연결됨
+++ 대화 참여자 수 : [3]
ㅇㅇㅇㅇㅇㅇ
이게 아닌가?
ㅇㅇㅇㅇㅇ
ㅇㅇㅇ
왜 한번에 안나오지
오 나오는군..
ㅇ그렇네요
/quit
+++ 대화 참여자 수 : [2]
[127.0.0.1] 접속 종료
/ㅂquioe
/quit
+++ 대화 참여자 수 : [1]
[127.0.0.1] 접속 종료
ㅂ
/벼ㅑㅅ
/quit
+++ 대화 참여자 수 : [0]
[127.0.0.1] 접속 종료
+++채팅 서버를 종료 합니다.


## 200 채팅 클라이언트 만들기
- 채팅 서버와 연결된 다수의 사용자와 채팅이 가능한 클라이언트를 생성

In [22]:
import socket
from threading import Thread

HOST = 'localhost'
PORT = 9009

def rcvmsg(sock):
    while True:
        try:
            data = sock.recv(1024)
            if not data:
                break
            print(data.decode())
        except:
            pass
        
def runchat():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((HOST, PORT))
        t = Thread(target=rcvmsg, args = (sock,))
        t.daemon = True
        t.start()
        
        while True:
            msg = input()
            if msg =='/quit':
                sock.send(msg.encode())
                break
            sock.send(msg.encode())
            
runchat()

ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다